# Reinventing Multi-Modal Search with Anyscale and MongoDB

What we are learning about and building today: https://www.anyscale.com/blog/reinventing-multi-modal-search-with-anyscale-and-mongodb

The following instructions will help you get set up your environment

## Register or login to Anyscale

If you don't have an Anyscale account, you can register [here](https://console.anyscale.com/register/ha?utm_source=github&utm_medium=github&utm_content=multi-modal-search-anyscale-mongodb).

If you already have an account, [login](https://console.anyscale.com/v2?utm_source=github&utm_medium=github&utm_content=multi-modal-search-anyscale-mongodb) here.

## Register or login to MongoDB

If you don't have a MongoDB account, follow the instructions [here](https://www.mongodb.com/docs/guides/atlas/account/) to sign up for a free MongoDB Atlas account.

## Create a free MongoDB cluster

If you don't have a MongoDB cluster created, follow the instructions [here](https://www.mongodb.com/docs/guides/atlas/cluster/) to create a free MongoDB Atlas cluster. __Name it `myntra`__

Use the `0.0.0.0/0` network access list to allow public access to your cluster. To do so follow the instructions [here](https://www.mongodb.com/docs/guides/atlas/network-connections/).

Create your database user and save the password.

Get the connection string for your MongoDB cluster by following the instructions [here](https://www.mongodb.com/docs/guides/atlas/connection-string/).

## Register or login to Hugging Face

If you don't have a Hugging Face account, you can register [here](https://huggingface.co/join). 

If you already have an account, [login](https://huggingface.co/login) here.

Visit the [tokens](https://huggingface.co/settings/tokens) page to generate a new API token.

Visit the following model pages and request access to these models:
- [llava-hf/llava-v1.6-mistral-7b-hf](https://huggingface.co/llava-hf/llava-v1.6-mistral-7b-hf)
- [mistralai/Mistral-7B-Instruct-v0.1](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1)

Once you have access to these models, you can proceed with the next steps.

## Launch a workspace in Anyscale for this project

Choose the Anyscale Ray Summit 2024 template

## Configure environment variables in your Anyscale Workspace

Under the __Dependencies__ tab in the workspace view, set the MongoDB connection string `DB_CONNECTION_STRING` and huggingface access token `HF_TOKEN` as environment variables.

<img src="https://anyscale-public-materials.s3.us-west-2.amazonaws.com/mongodb-demo/screenshots/workspace-dependencies.png" width="800px" alt="env-vars-setup-workspace">

---

## Create a vector search index in MongoDB

To do so, follow these steps

1. Select your database
2. Click on the "Search Indexes" tab or the "Atlas Search" tab

<img src="https://anyscale-public-materials.s3.us-west-2.amazonaws.com/mongodb-demo/screenshots/create_index_manual.png" width="800px" alt="create-index-manual"/>

3. Click on the "Create a Search index" button

#### From here on, follow these steps to build the Atlas Vector Search Index

1. Click on the "JSON Editor" Option

<img src="https://anyscale-public-materials.s3.us-west-2.amazonaws.com/mongodb-demo/screenshots/atlas_vector_search_json_editor_selected.png" width="800px" alt="json-editor-selected"/>

2. Click Next

3. Copy this JSON

```JSON
{ "fields": [
      {
          "numDimensions": 1024,
          "similarity": "cosine",
          "type": "vector",
          "path": "description_embedding"
      },
      {
          "numDimensions": 1024,
          "similarity": "cosine",
          "type": "vector",
          "path": "name_embedding"
      },                            
      {
          "type": "filter",
          "path": "category"
      },
      {
          "type": "filter",
          "path": "season"
      },
      {
          "type": "filter",
          "path": "color"
      },
      {
          "type": "filter",
          "path": "rating"
      },
      {
          "type": "filter",
          "path": "price"
      }
  ]
}
```

#### Ensure the index is called `vector_search_index`

4. Select your collection in the left-hand menu, paste the JSON into the text area, and click Next

<img src="https://anyscale-public-materials.s3.us-west-2.amazonaws.com/mongodb-demo/screenshots/atlas_vector_search_json_editor_full.png" width="800px" alt="json-editor-full">

---

## Create a full-text search index in MongoDB

1. Click on the "JSON Editor" Option

<img src="https://anyscale-public-materials.s3.us-west-2.amazonaws.com/mongodb-demo/screenshots/atlas_search_json_editor_selected.png" width="800px" alt="json-editor-selected">

2. Click Next

3. Copy the JSON

```JSON
{
    "mappings": {
        "dynamic": false,
        "fields": {
            "name": {
                "type": "string",
                "analyzer": "lucene.standard"
            }
        }
    }
}
```

#### Ensure the index is called `lexical_text_search_index`

4. Select your collection in the left-hand menu, paste the JSON, and click Next

<img src="https://anyscale-public-materials.s3.us-west-2.amazonaws.com/mongodb-demo/screenshots/atlas_search_json_editor_full.png" width="800px" alt="json-editor-full">

# Architecture

We split our system into an offline data indexing stage and an online search stage.

The offline data indexing stage performs the processing, embedding, and upserting text and images into a MongoDB database that supports vector search across multiple fields and dimensions. This stage is built by running multi-modal data pipelines at scale using Anyscale for AI compute platform.

The online search stage performs the necessary search operations by combining legacy text matching with advanced semantic search capabilities offered by MongoDB. This stage is built by running a multi-modal search backend on Anyscale.

## Multi-Modal Data Pipelines at Scale

### Overview
The data pipelines show how to perform offline batch inference and embeddings generation at scale. The pipelines are designed to handle both text and image data by running multi-modal large language model instances. 

### Technology Stack

- `ray[data]`
- `vLLM`
- `pymongo`
- `sentence-transformers`

## Multi-Modal Search at Scale

### Overview
The search backend combines legacy lexical text matching with advanced semantic search capabilities, offering a robust hybrid search solution. 

### Technology Stack
- `ray[serve]`
- `gradio`
- `motor`
- `sentence-transformers`

## Test our connection

In [ ]:
import pymongo
from pymongo import MongoClient, ASCENDING, DESCENDING
import os
from pymongo.operations import IndexModel

In [ ]:
db_name: str = "myntra"
collection_name: str = "myntra-items-offline"

In [ ]:
client = MongoClient(os.environ["DB_CONNECTION_STRING"])
db = client[db_name]

*If the `DB_CONNECTION_STRING` env var is not found, you may need to terminate and then restart the workspace.*

### Setup collection

Run this code one time after you've created your database, to set up the collection.

In [ ]:
db.drop_collection(collection_name)

my_collection = db[collection_name]

my_collection.create_indexes(
    [
        IndexModel([("rating", DESCENDING)]),
        IndexModel([("category", ASCENDING)]),
        IndexModel([("season", ASCENDING)]),
        IndexModel([("color", ASCENDING)]),
    ]
)

### Count docs

In [ ]:
my_collection = db[collection_name]

In [ ]:
my_collection.count_documents({})

### Empty collection

As you're working, you may have experiment, errors, or changes which alter the MongoDB collection. To drop all records in the collection, use the following line.

In [ ]:
my_collection.delete_many({})